# Recurrent neural networks for H.P Lovecraft text generation

"The color out of space" is one of my favorite tales from Lovecraft, i will use it(as well as others as the call of cthulhu) to create a recurrent neural network in tensorflow that learns his style and generates new text in his style

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [2]:
with open('colour_out_of_space.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [3]:
text

'THE COLOUR OUT OF SPACE\n\nWest of Arkham the hills rise wild, and there are valleys with deep woods that no axe has ever cut. There are dark narrow glens where the trees slope fantastically, and where thin brooklets trickle without ever having caught the glint of sunlight. On the gentle slopes there are farms, ancient and rocky, with squat, moss-coated cottages brooding eternally over old New England secrets in the lee of great ledges; but these are all vacant now, the wide chimneys crumbling and the shingled sides bulging perilously beneath low gambrel roofs.\n\nThe old folk have gone away, and foreigners do not like to live there. French-Canadians have tried it, Italians have tried it, and the Poles have come and departed. It is not because of anything that can be seen or heard or handled, but because of something that is imagined. The place is not good for imagination, and does not bring restful dreams at night. It must be this which keeps the foreigners away, for old Ammi Pierce 

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [4]:
text[:100]

'THE COLOUR OUT OF SPACE\n\nWest of Arkham the hills rise wild, and there are valleys with deep woods t'

And we can see the characters encoded as integers.

In [5]:
encoded[:100]

array([26, 13, 22, 41, 18, 50, 58, 50, 67, 57, 41, 50, 67, 26, 41, 50, 54,
       41, 33, 27, 74, 18, 22,  9,  9, 56, 42, 45, 47, 41, 21, 61, 41, 74,
       69, 40, 36,  5, 10, 41, 47, 36, 42, 41, 36, 15, 48, 48, 45, 41, 69,
       15, 45, 42, 41,  8, 15, 48, 11, 66, 41,  5,  3, 11, 41, 47, 36, 42,
       69, 42, 41,  5, 69, 42, 41, 23,  5, 48, 48, 42, 52, 45, 41,  8, 15,
       47, 36, 41, 11, 42, 42, 37, 41,  8, 21, 21, 11, 45, 41, 47], dtype=int32)

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [6]:
len(vocab)

77

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>
We have our text encoded as integers as one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the number of batches we can make from some array `arr`, you divide the length of `arr` by the batch size. Once you know the number of batches and the batch size, you can get the total number of characters to keep.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`n_seqs` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$ where $K$ is the number of batches.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. You'll usually see the first input character used as the last target character, so something like this:
```python
y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
```
where `x` is the input batch and `y` is the target batch.

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

> **Exercise:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [7]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the batch size and number of batches we can make
    batch_size = n_seqs * n_steps 
    n_batches =  len(arr)//batch_size
    
    # Keep only enough characters to make full batches
    arr =  arr[:n_batches*batch_size]
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs,-1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:,n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros(x.shape)
        y[:,:-1],y[:,-1] = x[:,1:] ,x[:,0]
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [8]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [9]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[26 13 22 41 18 50 58 50 67 57]
 [41  8 36 15 38 36 41 69 42 45]
 [42 15  3  7 41 11 69  5 15  3]
 [22 23 42 69 52 41 47 69 15 61]
 [41 47 36  5 47 41 47 69 42 42]
 [ 5 38 40  8 21 21 11  9  9 46]
 [23 42 41  7 48 15 10 37 45 42]
 [11 41 47 36  5 47 41 35  5 47]
 [47 36 41 10  5 40 42 45 41 23]
 [48 15 23 42 41 47 36 42 41  3]]

y
 [[ 13.  22.  41.  18.  50.  58.  50.  67.  57.  41.]
 [  8.  36.  15.  38.  36.  41.  69.  42.  45.  42.]
 [ 15.   3.   7.  41.  11.  69.   5.  15.   3.  42.]
 [ 23.  42.  69.  52.  41.  47.  69.  15.  61.  48.]
 [ 47.  36.   5.  47.  41.  47.  69.  42.  42.  41.]
 [ 38.  40.   8.  21.  21.  11.   9.   9.  46.   2.]
 [ 42.  41.   7.  48.  15.  10.  37.  45.  42.  45.]
 [ 41.  47.  36.   5.  47.  41.  35.   5.  47.  70.]
 [ 36.  41.  10.   5.  40.  42.  45.  41.  23.  15.]
 [ 15.  23.  42.  41.  47.  36.  42.  41.   3.   5.]]


In [10]:
x

array([[26, 13, 22, 41, 18, 50, 58, 50, 67, 57, 41, 50, 67, 26, 41, 50, 54,
        41, 33, 27, 74, 18, 22,  9,  9, 56, 42, 45, 47, 41, 21, 61, 41, 74,
        69, 40, 36,  5, 10, 41, 47, 36, 42, 41, 36, 15, 48, 48, 45, 41],
       [41,  8, 36, 15, 38, 36, 41, 69, 42, 45, 42, 10, 35, 48, 42, 11, 41,
        45, 21, 10, 42, 41, 21, 61, 41, 47, 36, 42, 41, 35,  5,  3, 11, 45,
        41, 15,  3, 41, 47, 36, 42, 41, 10, 42, 47, 42, 21, 69, 71, 45],
       [42, 15,  3,  7, 41, 11, 69,  5, 15,  3, 42, 11, 41, 21, 61, 41, 45,
        21, 10, 42, 47, 36, 15,  3,  7, 70, 70, 45, 21, 10, 42, 47, 36, 15,
         3,  7, 41,  8,  5, 45, 41, 61,  5, 45, 47, 42,  3, 15,  3,  7],
       [22, 23, 42, 69, 52, 41, 47, 69, 15, 61, 48, 42, 41, 21, 61, 41, 47,
        36, 42, 41, 45, 38, 42,  3, 42, 41, 35, 20, 69,  3, 42, 11, 41, 15,
        47, 45, 42, 48, 61, 41, 15,  3, 47, 21, 41, 36, 15, 45, 41, 35],
       [41, 47, 36,  5, 47, 41, 47, 69, 42, 42, 41, 47, 21, 37, 41, 36, 42,
        15,  7, 36, 47, 

In [11]:
y

array([[ 13.,  22.,  41.,  18.,  50.,  58.,  50.,  67.,  57.,  41.,  50.,
         67.,  26.,  41.,  50.,  54.,  41.,  33.,  27.,  74.,  18.,  22.,
          9.,   9.,  56.,  42.,  45.,  47.,  41.,  21.,  61.,  41.,  74.,
         69.,  40.,  36.,   5.,  10.,  41.,  47.,  36.,  42.,  41.,  36.,
         15.,  48.,  48.,  45.,  41.,  26.],
       [  8.,  36.,  15.,  38.,  36.,  41.,  69.,  42.,  45.,  42.,  10.,
         35.,  48.,  42.,  11.,  41.,  45.,  21.,  10.,  42.,  41.,  21.,
         61.,  41.,  47.,  36.,  42.,  41.,  35.,   5.,   3.,  11.,  45.,
         41.,  15.,   3.,  41.,  47.,  36.,  42.,  41.,  10.,  42.,  47.,
         42.,  21.,  69.,  71.,  45.,  41.],
       [ 15.,   3.,   7.,  41.,  11.,  69.,   5.,  15.,   3.,  42.,  11.,
         41.,  21.,  61.,  41.,  45.,  21.,  10.,  42.,  47.,  36.,  15.,
          3.,   7.,  70.,  70.,  45.,  21.,  10.,  42.,  47.,  36.,  15.,
          3.,   7.,  41.,   8.,   5.,  45.,  41.,  61.,   5.,  45.,  47.,
         42.,   3.,  1

If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`. This will be a scalar, that is a 0-D tensor. To make a scalar, you create a placeholder without giving it a size.

> **Exercise:** Create the input placeholders in the function below.

In [12]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32,[batch_size,num_steps],name="inputs")
    targets = tf.placeholder(tf.int32,[batch_size,num_steps],name="targets")
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32,name="keep_prob")
    
    return inputs, targets, keep_prob

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

We first create a basic LSTM cell with

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

where `num_units` is the number of units in the hidden layers in the cell. Then we can add dropout by wrapping it with 

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
You pass in a cell and it will automatically add dropout to the inputs or outputs. Finally, we can stack up the LSTM cells into layers with [`tf.contrib.rnn.MultiRNNCell`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell). With this, you pass in a list of cells and it will send the output of one cell into the next cell. For example,

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

This might look a little weird if you know Python well because this will create a list of the same `cell` object. However, TensorFlow will create different weight matrices for all `cell` objects. Even though this is actually multiple LSTM cells stacked on each other, you can treat the multiple layers as one cell.

We also need to create an initial cell state of all zeros. This can be done like so

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

> **Exercise:** Below, implement the `build_lstm` function to create these LSTM cells and the initial state.

In [13]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    # Use a basic LSTM cell
    # Add dropout to the cell outputs
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper( tf.contrib.rnn.BasicLSTMCell(lstm_size)) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size,tf.float32)
    
    return cell, initial_state

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character, so we want this layer to have size $C$, the number of classes/characters we have in our text.

If our input has batch size $N$, number of steps $M$, and the hidden layer has $L$ hidden units, then the output is a 3D tensor with size $N \times M \times L$. The output of each LSTM cell has size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total size is $N \times M \times L$. 

We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make things easier, we should reshape the outputs into a 2D tensor with shape $(M * N) \times L$. That is, one row for each sequence and step, where the values of each row are the output from the LSTM cells. We get the LSTM output as a list, `lstm_output`. First we need to concatenate this whole list into one array with [`tf.concat`](https://www.tensorflow.org/api_docs/python/tf/concat). Then, reshape it (with `tf.reshape`) to size $(M * N) \times L$.

One we have the outputs reshaped, we can do the matrix multiplication with the weights. We need to wrap the weight and bias variables in a variable scope with `tf.variable_scope(scope_name)` because there are weights being created in the LSTM cells. TensorFlow will throw an error if the weights created here have the same names as the weights created in the LSTM cells, which they will be default. To avoid this, we wrap the variables in a variable scope so we can give them unique names.

> **Exercise:** Implement the output layer in the function below.

In [14]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        lstm_output: List of output tensors from the LSTM layer
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # Concatenate lstm_output over axis 1 (the columns)
    seq_output = tf.concat(lstm_output,axis=1)
    # Reshape seq_output to a 2D tensor with lstm_size columns
    x = tf.reshape(seq_output,[-1,in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        # Create the weight and bias variables here
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size),stddev=0.1))
        softmax_b = tf.Variable(tf.zeros([out_size]))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits =  tf.add(tf.matmul(x,softmax_w),softmax_b) 
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits,name ="out")
    print(out)
    return out, logits

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the one-hot targets so it's a 2D tensor with size $(M*N) \times C$ where $C$ is the number of classes/characters we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with $C$ units. So our logits will also have size $(M*N) \times C$.

Then we run the logits and targets through `tf.nn.softmax_cross_entropy_with_logits` and find the mean to get the loss.

>**Exercise:** Implement the loss calculation in the function below.

In [15]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    # One-hot encode targets and reshape to match logits, one row per sequence per step
    y_one_hot = tf.one_hot(targets,num_classes)
    y_reshaped =  tf.reshape(y_one_hot,logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y_reshaped))
    
    return loss

### Optimizer

Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

In [16]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

> **Exercise:** Use the functions you've implemented previously and `tf.nn.dynamic_rnn` to build the network.

In [17]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size,num_steps)
        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size,num_layers,batch_size,self.keep_prob)
        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs,num_classes)
        
        # Run each sequence step through the RNN with tf.nn.dynamic_rnn 
        outputs, state = tf.nn.dynamic_rnn(cell,x_one_hot,initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs,lstm_size,num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss =  build_loss(self.logits,self.targets,lstm_size,num_classes)
        self.optimizer = build_optimizer(self.loss,learning_rate,grad_clip)

## Hyperparameters

Here are the hyperparameters for the network.

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

In [71]:
batch_size = 100         # Sequences per batch
num_steps = 500          # Number of sequence steps per batch
lstm_size = 600         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001    # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}.ckpt`

> **Exercise:** Set the hyperparameters above to train the network. Watch the training loss, it should be consistently dropping. Also, I highly advise running this on a GPU.

In [74]:
epochs = 1000
# Save every N iterations
save_every_n = 200
with tf.device("/gpu:0"):
    model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Tensor("out:0", shape=(50000, 77), dtype=float32)
Epoch: 1/1000...  Training Step: 1...  Training loss: 4.3408...  1.0878 sec/batch
Epoch: 1/1000...  Training Step: 2...  Training loss: 4.2082...  1.0007 sec/batch
Epoch: 2/1000...  Training Step: 3...  Training loss: 4.1279...  1.0007 sec/batch
Epoch: 2/1000...  Training Step: 4...  Training loss: 3.7121...  1.0017 sec/batch
Epoch: 3/1000...  Training Step: 5...  Training loss: 3.5425...  1.0019 sec/batch
Epoch: 3/1000...  Training Step: 6...  Training loss: 3.3515...  1.0036 sec/batch
Epoch: 4/1000...  Training Step: 7...  Training loss: 3.1737...  1.0029 sec/batch
Epoch: 4/1000...  Training Step: 8...  Training loss: 3.0954...  1.0030 sec/batch
Epoch: 5/1000...  Training Step: 9...  Training loss: 3.1594...  1.0037 sec/batch
Epoch: 5/1000...  Training Step: 10...  Training loss: 3.0825...  1.0034 sec/batch
Epoch: 6/1000...  Training Step: 11...  Training loss: 3.0662...  1.0035 sec/batch
Epoch: 6/1000...  Training Step: 12...  Traini

Epoch: 50/1000...  Training Step: 99...  Training loss: 2.8565...  1.0080 sec/batch
Epoch: 50/1000...  Training Step: 100...  Training loss: 2.8350...  1.0085 sec/batch
Epoch: 51/1000...  Training Step: 101...  Training loss: 2.8367...  1.0085 sec/batch
Epoch: 51/1000...  Training Step: 102...  Training loss: 2.8177...  1.0082 sec/batch
Epoch: 52/1000...  Training Step: 103...  Training loss: 2.8207...  1.0081 sec/batch
Epoch: 52/1000...  Training Step: 104...  Training loss: 2.7981...  1.0078 sec/batch
Epoch: 53/1000...  Training Step: 105...  Training loss: 2.7869...  1.0080 sec/batch
Epoch: 53/1000...  Training Step: 106...  Training loss: 2.7884...  1.0083 sec/batch
Epoch: 54/1000...  Training Step: 107...  Training loss: 3.0576...  1.0082 sec/batch
Epoch: 54/1000...  Training Step: 108...  Training loss: 2.8580...  1.0086 sec/batch
Epoch: 55/1000...  Training Step: 109...  Training loss: 2.8351...  1.0080 sec/batch
Epoch: 55/1000...  Training Step: 110...  Training loss: 2.7999...

Epoch: 98/1000...  Training Step: 196...  Training loss: 2.3413...  1.0107 sec/batch
Epoch: 99/1000...  Training Step: 197...  Training loss: 2.3448...  1.0105 sec/batch
Epoch: 99/1000...  Training Step: 198...  Training loss: 2.3298...  1.0110 sec/batch
Epoch: 100/1000...  Training Step: 199...  Training loss: 2.3454...  1.0111 sec/batch
Epoch: 100/1000...  Training Step: 200...  Training loss: 2.3284...  1.0109 sec/batch
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'dict' object has no attribute 'name'
Epoch: 101/1000...  Training Step: 201...  Training loss: 2.3313...  1.0094 sec/batch
Epoch: 101/1000...  Training Step: 202...  Training loss: 2.3283...  1.0112 sec/batch
Epoch: 102/1000...  Training Step: 203...  Training loss: 2.3229...  1.0106 sec/batch
Epoch: 102/1000...  Training Step: 204...  Training loss: 2.3189...  1.0109 sec/batch
Epoch: 103/1000...  Training Step: 205...  Training loss: 2.3200...  1.0104 sec/batch
Epoch: 103/1000..

Epoch: 145/1000...  Training Step: 290...  Training loss: 2.1577...  1.0176 sec/batch
Epoch: 146/1000...  Training Step: 291...  Training loss: 2.1628...  1.0107 sec/batch
Epoch: 146/1000...  Training Step: 292...  Training loss: 2.1506...  1.0111 sec/batch
Epoch: 147/1000...  Training Step: 293...  Training loss: 2.1523...  1.0105 sec/batch
Epoch: 147/1000...  Training Step: 294...  Training loss: 2.1466...  1.0107 sec/batch
Epoch: 148/1000...  Training Step: 295...  Training loss: 2.1451...  1.0110 sec/batch
Epoch: 148/1000...  Training Step: 296...  Training loss: 2.1356...  1.0111 sec/batch
Epoch: 149/1000...  Training Step: 297...  Training loss: 2.1413...  1.0108 sec/batch
Epoch: 149/1000...  Training Step: 298...  Training loss: 2.1322...  1.0112 sec/batch
Epoch: 150/1000...  Training Step: 299...  Training loss: 2.1300...  1.0107 sec/batch
Epoch: 150/1000...  Training Step: 300...  Training loss: 2.1236...  1.0117 sec/batch
Epoch: 151/1000...  Training Step: 301...  Training lo

Epoch: 193/1000...  Training Step: 386...  Training loss: 1.9351...  1.0094 sec/batch
Epoch: 194/1000...  Training Step: 387...  Training loss: 1.9352...  1.0100 sec/batch
Epoch: 194/1000...  Training Step: 388...  Training loss: 1.9304...  1.0093 sec/batch
Epoch: 195/1000...  Training Step: 389...  Training loss: 1.9296...  1.0094 sec/batch
Epoch: 195/1000...  Training Step: 390...  Training loss: 1.9232...  1.0097 sec/batch
Epoch: 196/1000...  Training Step: 391...  Training loss: 1.9229...  1.0092 sec/batch
Epoch: 196/1000...  Training Step: 392...  Training loss: 1.9189...  1.0093 sec/batch
Epoch: 197/1000...  Training Step: 393...  Training loss: 1.9252...  1.0094 sec/batch
Epoch: 197/1000...  Training Step: 394...  Training loss: 1.9327...  1.0099 sec/batch
Epoch: 198/1000...  Training Step: 395...  Training loss: 1.9663...  1.0100 sec/batch
Epoch: 198/1000...  Training Step: 396...  Training loss: 1.9256...  1.0096 sec/batch
Epoch: 199/1000...  Training Step: 397...  Training lo

...

Epoch: 804/1000...  Training Step: 1607...  Training loss: 0.0143...  1.0095 sec/batch
Epoch: 804/1000...  Training Step: 1608...  Training loss: 0.0057...  1.0107 sec/batch
Epoch: 805/1000...  Training Step: 1609...  Training loss: 0.0142...  1.0106 sec/batch
Epoch: 805/1000...  Training Step: 1610...  Training loss: 0.0062...  1.0103 sec/batch
Epoch: 806/1000...  Training Step: 1611...  Training loss: 0.0141...  1.0097 sec/batch
Epoch: 806/1000...  Training Step: 1612...  Training loss: 0.0055...  1.0097 sec/batch
Epoch: 807/1000...  Training Step: 1613...  Training loss: 0.0140...  1.0102 sec/batch
Epoch: 807/1000...  Training Step: 1614...  Training loss: 0.0054...  1.0107 sec/batch
Epoch: 808/1000...  Training Step: 1615...  Training loss: 0.0139...  1.0098 sec/batch
Epoch: 808/1000...  Training Step: 1616...  Training loss: 0.0051...  1.0099 sec/batch
Epoch: 809/1000...  Training Step: 1617...  Training loss: 0.0137...  1.0096 sec/batch
Epoch: 809/1000...  Training Step: 1618... 

Epoch: 851/1000...  Training Step: 1702...  Training loss: 0.0032...  1.0101 sec/batch
Epoch: 852/1000...  Training Step: 1703...  Training loss: 0.0117...  1.0107 sec/batch
Epoch: 852/1000...  Training Step: 1704...  Training loss: 0.0032...  1.0103 sec/batch
Epoch: 853/1000...  Training Step: 1705...  Training loss: 0.0116...  1.0095 sec/batch
Epoch: 853/1000...  Training Step: 1706...  Training loss: 0.0032...  1.0098 sec/batch
Epoch: 854/1000...  Training Step: 1707...  Training loss: 0.0116...  1.0099 sec/batch
Epoch: 854/1000...  Training Step: 1708...  Training loss: 0.0031...  1.0095 sec/batch
Epoch: 855/1000...  Training Step: 1709...  Training loss: 0.0116...  1.0096 sec/batch
Epoch: 855/1000...  Training Step: 1710...  Training loss: 0.0031...  1.0102 sec/batch
Epoch: 856/1000...  Training Step: 1711...  Training loss: 0.0116...  1.0147 sec/batch
Epoch: 856/1000...  Training Step: 1712...  Training loss: 0.0031...  1.0118 sec/batch
Epoch: 857/1000...  Training Step: 1713... 

Epoch: 899/1000...  Training Step: 1797...  Training loss: 0.0105...  1.0104 sec/batch
Epoch: 899/1000...  Training Step: 1798...  Training loss: 0.0026...  1.0096 sec/batch
Epoch: 900/1000...  Training Step: 1799...  Training loss: 0.0105...  1.0101 sec/batch
Epoch: 900/1000...  Training Step: 1800...  Training loss: 0.0026...  1.0089 sec/batch
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'dict' object has no attribute 'name'
Epoch: 901/1000...  Training Step: 1801...  Training loss: 0.0105...  1.0088 sec/batch
Epoch: 901/1000...  Training Step: 1802...  Training loss: 0.0026...  1.0095 sec/batch
Epoch: 902/1000...  Training Step: 1803...  Training loss: 0.0105...  1.0090 sec/batch
Epoch: 902/1000...  Training Step: 1804...  Training loss: 0.0025...  1.0100 sec/batch
Epoch: 903/1000...  Training Step: 1805...  Training loss: 0.0105...  1.0098 sec/batch
Epoch: 903/1000...  Training Step: 1806...  Training loss: 0.0025...  1.0091 sec/batch
Epoc

Epoch: 945/1000...  Training Step: 1889...  Training loss: 0.0105...  1.0095 sec/batch
Epoch: 945/1000...  Training Step: 1890...  Training loss: 0.0028...  1.0101 sec/batch
Epoch: 946/1000...  Training Step: 1891...  Training loss: 0.0104...  1.0104 sec/batch
Epoch: 946/1000...  Training Step: 1892...  Training loss: 0.0028...  1.0108 sec/batch
Epoch: 947/1000...  Training Step: 1893...  Training loss: 0.0103...  1.0088 sec/batch
Epoch: 947/1000...  Training Step: 1894...  Training loss: 0.0027...  1.0091 sec/batch
Epoch: 948/1000...  Training Step: 1895...  Training loss: 0.0103...  1.0103 sec/batch
Epoch: 948/1000...  Training Step: 1896...  Training loss: 0.0027...  1.0102 sec/batch
Epoch: 949/1000...  Training Step: 1897...  Training loss: 0.0102...  1.0106 sec/batch
Epoch: 949/1000...  Training Step: 1898...  Training loss: 0.0026...  1.0097 sec/batch
Epoch: 950/1000...  Training Step: 1899...  Training loss: 0.0102...  1.0101 sec/batch
Epoch: 950/1000...  Training Step: 1900... 

Epoch: 992/1000...  Training Step: 1984...  Training loss: 0.0019...  1.0110 sec/batch
Epoch: 993/1000...  Training Step: 1985...  Training loss: 0.0091...  1.0101 sec/batch
Epoch: 993/1000...  Training Step: 1986...  Training loss: 0.0019...  1.0112 sec/batch
Epoch: 994/1000...  Training Step: 1987...  Training loss: 0.0090...  1.0095 sec/batch
Epoch: 994/1000...  Training Step: 1988...  Training loss: 0.0019...  1.0097 sec/batch
Epoch: 995/1000...  Training Step: 1989...  Training loss: 0.0090...  1.0100 sec/batch
Epoch: 995/1000...  Training Step: 1990...  Training loss: 0.0019...  1.0101 sec/batch
Epoch: 996/1000...  Training Step: 1991...  Training loss: 0.0090...  1.0097 sec/batch
Epoch: 996/1000...  Training Step: 1992...  Training loss: 0.0019...  1.0106 sec/batch
Epoch: 997/1000...  Training Step: 1993...  Training loss: 0.0090...  1.0110 sec/batch
Epoch: 997/1000...  Training Step: 1994...  Training loss: 0.0019...  1.0099 sec/batch
Epoch: 998/1000...  Training Step: 1995... 

#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [75]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i2000_l600.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l600.ckpt"
all_model_checkpoint_paths: "checkpoints/i400_l600.ckpt"
all_model_checkpoint_paths: "checkpoints/i600_l600.ckpt"
all_model_checkpoint_paths: "checkpoints/i800_l600.ckpt"
all_model_checkpoint_paths: "checkpoints/i1000_l600.ckpt"
all_model_checkpoint_paths: "checkpoints/i1200_l600.ckpt"
all_model_checkpoint_paths: "checkpoints/i1400_l600.ckpt"
all_model_checkpoint_paths: "checkpoints/i1600_l600.ckpt"
all_model_checkpoint_paths: "checkpoints/i1800_l600.ckpt"
all_model_checkpoint_paths: "checkpoints/i2000_l600.ckpt"

In [21]:
print(tf.__version__)

1.1.0-rc2


## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [22]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [23]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [53]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints/i720_l600.ckpt'

In [77]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="moutains")
print(samp)

Tensor("out:0", shape=(1, 77), dtype=float32)
INFO:tensorflow:Restoring parameters from checkpoints/i2000_l600.ckpt
moutainst of felinaal ous monstoristir. Ind moner or had not stread bey a sound ig founted but the had stometh promped it whit dindianled skenising for that sty phill tementh was fomened chifed in had colour as a load or dark the pabelal torme. The nours whach inss doy, but promen astle the distunted and of ever the saveraly of enmalys an the nexters of the gadeable deally in a cunden agrimat of awayy. The lotser had beoon of the mey outhifif quitice on the oljorm was over Nahum ofadide abreas he roush and soulf the wald, coaling the fream of the stand was ead in in the water. The who scely hod ran the herr end that the felther woods menter and lames and the sainton was the imes thin near neadly s everyand moness the ed they of comenest the hoctous the seat coloul tofs ax reuled ascid. Nahum had leee but ghoun apon cormabey thit tromes out in fimearian to quite nough as a

In [26]:
checkpoint = 'checkpoints/i200_l550.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Tensor("out:0", shape=(1, 77), dtype=float32)
INFO:tensorflow:Restoring parameters from checkpoints/i200_l550.ckpt
Farres anl ons the andit tha the santhe thast on thend tired bothe coret on the satith wint thed thas and the thin sinle af ald bor orest of whand beringes and this woll tor al thered simid wimlad solist on ale the tee hhe the th minl then ond thare bote an thes, bas the se momed the thent ind tho se sithe tisto the cithint at thinge bed af and al the thes tas ther ont ord anterad, bortind. wol tore toad sant thase thiringe at thor wore whe sinte se the weth at tal chit tore tol wasthe angh tand, and on tha mat oftint talt the chetand. whe whis there tans an hhithare anterith sis af titil ce the sor an th ald women oudese and af the til oldind worins wath thas the wate and the the cathed whed ouss onthend af thareting thedith toled tin wanth the the the the whathed, wor an tire hare thon the thom thin thimed onthe bised, bassisd bethe then oud ar te the che the cislong whe

In [28]:
checkpoint = 'checkpoints/i2000_l550.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Tensor("out:0", shape=(1, 77), dtype=float32)
INFO:tensorflow:Restoring parameters from checkpoints/i2000_l550.ckpt
Farm of the wiles of the sudest to deport in the smork the spece becamed  folfrrd on the filites in monscriend whas was and const fexped incomelarch of this wat preemst and blank had secled in hir counds and bubling, and syomered agealse th sciencedi; and pheshec varieno had groued the reating thatien chawmy accinent desmitter and digan the proper of the conerung in the yorne partis afthrint whar an aghar at moush of the n arouthl and Nahum as om this  aster there, an and assed the neally sit of and south in the merfire yound sheat when the siomowing lafter theirs haid had been therred shomst-rigned inal the well. There were ablar. who stiend of stripking sied. That in sheal calten as he maying place-door sees times of the scrap, and which it was ronchented intruat. the the skneated yeach with the professors was to meated. He had now like their when con, but sky whal was,